In [15]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px
import plotly.graph_objs as go
from geopy.geocoders import Nominatim
import folium
from folium.plugins import HeatMap
from folium.plugins import FastMarkerCluster
from plotly import tools
import re
from plotly.offline import init_notebook_mode, plot, iplot
from wordcloud import WordCloud, STOPWORDS 
from warnings import filterwarnings
filterwarnings('ignore')

In [16]:
nashik = pd.read_csv('nashik.csv',index_col=0)

In [17]:
nashik['housetype'].replace({'Independent_house':'Independent house'},inplace=True)

In [18]:
clean_data = pd.read_csv('clean_data',index_col = 0)

In [19]:
clean_data.head(1)

,address,owners,housetype,house_condition,BHK,price,per_month_emi,total_sqft,cordinates,latitude,longitude
0,"Sheetal Vihar, Bhagwant Nagar, Dr.Homi Bhabha ...",Mahendra Kotwal,Apartment,old,3.0,75.0,39.72,1550.0,Sheetal Vihar,19.98796,73.78739


In [20]:
nashik.columns

Index(['address', 'owners', 'housetype', 'house_condition', 'BHK', 'price',
       'per_month_emi', 'total_sqft'],
      dtype='object')

In [21]:
result = pd.merge(clean_data,nashik,how='left',on=['address', 'owners', 'housetype', 'house_condition', 'BHK', 'price',
       'per_month_emi', 'total_sqft'])

In [22]:
result[result.duplicated()]

,address,owners,housetype,house_condition,BHK,price,per_month_emi,total_sqft,cordinates,latitude,longitude
11,"krishnabaug apartment,Nashik Road, Nashik",Suvarna,Apartment,old,2.0,28.0,14.83,810.0,Krishna Baug,19.94790,73.83178
12,"krishnabaug apartment,Nashik Road, Nashik",Suvarna,Apartment,old,2.0,28.0,14.83,810.0,Krishna Baug,19.94790,73.83178
13,"krishnabaug apartment,Nashik Road, Nashik",Suvarna,Apartment,old,2.0,28.0,14.83,810.0,Krishna Baug,19.94790,73.83178
46,"Nashik Road, Nashik",Prabhakar Vadnere,Apartment,old,2.0,28.0,14.83,775.0,"Nashik Road, Nashik, Maharashtra",19.95531,73.84089
54,"Nashik Road, Nashik",Prabhakar Vadnere,Apartment,old,2.0,28.0,14.83,775.0,"Nashik Road, Nashik, Maharashtra",19.95531,73.84089
...,...,...,...,...,...,...,...,...,...,...,...
6390,"Swami Samarth Nagar, Nashik Road,Nashik",Sajish Gangurde,Independent house,old,2.0,40.0,21.18,1400.0,"Swami Samarth Nagar, Mankar Mala, Nashik, Maha...",19.95539,73.85953
6391,"Swami Samarth Nagar, Nashik Road,Nashik",Sajish Gangurde,Independent house,old,2.0,40.0,21.18,1400.0,"Swami Samarth Nagar, Mankar Mala, Nashik, Maha...",19.95539,73.85953
6393,"Balram Nagar, Nashik",Sajish Gangurde,Independent house,old,4.0,45.0,23.83,1450.0,"Balram Nagar, Panchavati, Nashik, Maharashtra",20.02450,73.83583
6394,"Balram Nagar, Nashik",Sajish Gangurde,Independent house,old,4.0,45.0,23.83,1450.0,"Balram Nagar, Panchavati, Nashik, Maharashtra",20.02450,73.83583


In [23]:
result.drop_duplicates(inplace=True)

In [24]:
result[result.duplicated()]

,address,owners,housetype,house_condition,BHK,price,per_month_emi,total_sqft,cordinates,latitude,longitude


In [25]:
result.housetype.unique()

array(['Apartment', 'Independent house'], dtype=object)

In [26]:
result.isnull().sum()

address            0
owners             0
housetype          0
house_condition    0
BHK                0
price              0
per_month_emi      0
total_sqft         0
cordinates         0
latitude           0
longitude          0
dtype: int64

In [27]:
result.head(50)

,address,owners,housetype,house_condition,BHK,price,per_month_emi,total_sqft,cordinates,latitude,longitude
0,"Sheetal Vihar, Bhagwant Nagar, Dr.Homi Bhabha ...",Mahendra Kotwal,Apartment,old,3.0,75.00,39.72,1550.0,Sheetal Vihar,19.987960,73.787390
1,"Samraat Dream Citi, Samta Nagar, Nashik",Jiten Dadarkar,Apartment,old,2.0,41.00,21.71,1000.0,Samraat Dream Citi,19.982460,73.811840
2,"Suryaprakash Apartment,Nashik Road, Nashik",Pankaj,Apartment,old,2.0,53.35,28.25,970.0,Surya Prakash,19.975850,73.788750
3,"Adishvar Residency,Nashik Road, Nashik",Saurav,Apartment,old,2.0,55.00,29.13,1000.0,"Nashik Road, Vadner Dumala, Nashik, Maharashtr...",19.930532,73.811734
4,"Nashik Road, Nashik",Vivek Pavanaskar,Apartment,old,2.0,27.00,14.30,853.0,"Nashik Road, Nashik, Maharashtra",19.955310,73.840890
5,"Gurucharani,Nashik Road, Nashik",Darshan Muley,Apartment,old,2.0,42.00,22.24,1050.0,Gurucharni,20.001320,73.755030
6,"CDO-MERI Colony, Nashik",Prashant,Apartment,old,2.0,35.50,18.80,1035.0,"Meri Colony, Meri, Nashik, Maharashtra",20.026790,73.800240
7,"Ram Sneh Apartment,Nashik Road, Nashik",Sweety Surana,Apartment,old,3.0,48.00,25.42,1375.0,Ram Sneha,19.958350,73.825000
8,"gajanan pushp aptshiram nagar,Nashik Road, Nashik",Shriram Kulkarni,Apartment,old,2.0,32.00,16.95,850.0,"Ram Nagar, Sane Guruji Nagar, Nashik, Maharashtra",19.957080,73.834670
9,"B2 LAXMI RESIDENCY ,Nashik Road, Nashik",GAURAV Chaturvedi,Apartment,old,2.0,32.00,16.95,1020.0,Laxmi Residency,20.016760,73.793520


In [28]:
result.describe(include = 'all')

,address,owners,housetype,house_condition,BHK,price,per_month_emi,total_sqft,cordinates,latitude,longitude
count,4864,4864,4864,4864,4864.000000,4864.000000,4864.000000,4864.000000,4864,4864.000000,4864.000000
unique,1657,2615,2,2,NaN,NaN,NaN,NaN,610,NaN,NaN
top,"Nashik Road, Nashik",Mr Sanjay,Apartment,old,NaN,NaN,NaN,NaN,"Nashik Road, Nashik, Maharashtra",NaN,NaN
freq,260,164,3935,3180,NaN,NaN,NaN,NaN,284,NaN,NaN
mean,NaN,NaN,NaN,NaN,1.968030,36.812921,19.939922,1020.905016,NaN,20.003973,73.833545
std,NaN,NaN,NaN,NaN,0.813694,20.096963,10.041486,328.183847,NaN,0.390636,0.575925
min,NaN,NaN,NaN,NaN,1.000000,6.090000,2.970000,355.000000,NaN,17.353537,70.761970
25%,NaN,NaN,NaN,NaN,1.000000,24.702500,13.240000,850.000000,NaN,19.959310,73.759400
50%,NaN,NaN,NaN,NaN,2.000000,32.650000,18.000000,975.000000,NaN,19.981484,73.782900
75%,NaN,NaN,NaN,NaN,2.000000,43.000000,23.830000,1100.750000,NaN,20.012770,73.814700


In [29]:
result.to_csv('result2.csv')

In [35]:
from impyute.imputation.cs import mice

In [36]:
from impyute.imputation.cs import mice
imputed = mice(df[['BHK', 'price',
       'per_month_emi', 'total_sqft']].values)

In [37]:
df = pd.read_csv('nashik.csv',index_col=0)

In [38]:
new_data = pd.DataFrame(imputed,columns=['BHK', 'price',
       'per_month_emi', 'total_sqft'])

In [39]:
df1 = df[['address', 'owners', 'housetype', 'house_condition']]
frames = [df1,new_data]
nashik = pd.concat(frames,axis=1)

In [40]:
nashik.describe()

,BHK,price,per_month_emi,total_sqft
count,5497.000000,5497.000000,5497.000000,5497.000000
mean,2.014574,55.079614,21.041009,1172.008990
std,0.905283,268.858305,22.422072,2282.780126
min,0.094934,1.000000,1.050000,150.000000
25%,1.000000,23.900000,12.247591,780.000000
50%,2.000000,38.000000,18.530000,962.000000
75%,2.000000,55.000000,24.940000,1212.000000
max,10.000000,9668.000000,844.000000,78785.775202


In [41]:
nashik.isnull().sum()

address             0
owners             13
housetype           0
house_condition     0
BHK                 0
price               0
per_month_emi       0
total_sqft          0
dtype: int64

In [4]:
nashik

,address,owners,housetype,house_condition,BHK,price,per_month_emi,total_sqft
0,"Sheetal Vihar, Bhagwant Nagar, Dr.Homi Bhabha ...",Mahendra Kotwal,Apartment,old,3.0,75.00,39.72,1550.0
1,"Samraat Dream Citi, Samta Nagar, Nashik",Jiten Dadarkar,Apartment,old,2.0,41.00,21.71,1000.0
2,"Suryaprakash Apartment,Nashik Road, Nashik",Pankaj,Apartment,old,2.0,53.35,28.25,970.0
3,"Adishvar Residency,Nashik Road, Nashik",Saurav,Apartment,old,2.0,55.00,29.13,1000.0
4,"Nashik Road, Nashik",Vivek Pavanaskar,Apartment,old,2.0,27.00,14.30,853.0
...,...,...,...,...,...,...,...,...
5492,"Vasantdadanagar, Adgaon,Nashik",Geeta Buildcon,Independent_house,old,2.0,NaN,NaN,NaN
5493,"Samarth Nagar, Makhmalabad,Nashik",Atharvashree Buildmat Private Limited,Independent_house,old,2.0,34.32,18.17,NaN
5494,"Bhor Town Ship, Nashik",Shree Prakash Developers,Independent_house,old,NaN,NaN,NaN,NaN
5495,"Chandshi, Nashik",Nirman Group,Independent_house,old,NaN,NaN,NaN,NaN


In [43]:
from geopy.geocoders import ArcGIS

In [44]:
nom=ArcGIS()

In [15]:
result.describe(include= 'all')

,address,owners,housetype,house_condition,BHK,price,per_month_emi,total_sqft,cordinates,latitude,longitude
count,6781,6768,6781,6781,6618.000000,5827.000000,5821.000000,5269.000000,4975,4975.000000,4975.000000
unique,1758,2791,2,2,NaN,NaN,NaN,NaN,590,NaN,NaN
top,"Swami Samarth Nagar, Nashik Road,Nashik",Sajish Gangurde,Apartment,old,NaN,NaN,NaN,NaN,"Swami Samarth Nagar, Mankar Mala, Nashik, Maha...",NaN,NaN
freq,800,1110,5543,4837,NaN,NaN,NaN,NaN,809,NaN,NaN
mean,NaN,NaN,NaN,NaN,2.019719,46.958471,20.064135,1057.425887,NaN,20.005498,73.840796
std,NaN,NaN,NaN,NaN,0.882227,261.655297,21.942646,776.403678,NaN,0.401922,0.594790
min,NaN,NaN,NaN,NaN,1.000000,1.000000,1.050000,150.000000,NaN,17.353537,70.761970
25%,NaN,NaN,NaN,NaN,1.000000,17.280000,11.650000,735.000000,NaN,19.955390,73.762350
50%,NaN,NaN,NaN,NaN,2.000000,28.000000,16.420000,912.000000,NaN,19.974750,73.793970
75%,NaN,NaN,NaN,NaN,2.000000,45.000000,23.830000,1200.000000,NaN,20.002950,73.840890


In [13]:
result.duplicated()

0       False
1       False
2       False
3       False
4       False
        ...  
6776    False
6777    False
6778    False
6779    False
6780    False
Length: 5211, dtype: bool

In [45]:
nashik1 =nashik.iloc[0:250]
nashik1['cordinates']= nashik1['address'].apply(nom.geocode)
nashik1['latitude'] = nashik1['cordinates'].apply(lambda x:x.latitude)
nashik1['longitude'] = nashik1['cordinates'].apply(lambda x:x.longitude)

GeocoderUnavailable: HTTPSConnectionPool(host='geocode.arcgis.com', port=443): Max retries exceeded with url: /arcgis/rest/services/World/GeocodeServer/findAddressCandidates?singleLine=Loksahakar+Nagar%2C+Nashik&f=json&maxLocations=1 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x00000251F1101550>, 'Connection to geocode.arcgis.com timed out. (connect timeout=1)'))

In [ ]:
nashik1.to_csv('nashik1.csv')

In [ ]:
nashik2 =nashik.iloc[500:1000]
nashik2['cordinates']= nashik2['address'].apply(nom.geocode)
nashik2['latitude'] = nashik2['cordinates'].apply(lambda x:x.latitude)
nashik2['longitude'] = nashik2['cordinates'].apply(lambda x:x.longitude)

In [ ]:
nashik2.to_csv('nashik2.csv')

In [11]:
nashik3 = nashik.iloc[1000:1500]
nashik3['cordinates']= nashik3['address'].apply(nom.geocode)


nashik3['latitude'] = nashik3['cordinates'].apply(lambda x:x.latitude)
nashik3['longitude'] = nashik3['cordinates'].apply(lambda x:x.longitude)


GeocoderUnavailable: HTTPSConnectionPool(host='geocode.arcgis.com', port=443): Max retries exceeded with url: /arcgis/rest/services/World/GeocodeServer/findAddressCandidates?singleLine=CDO-MERI+Colony%2C+Nashik&f=json&maxLocations=1 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001F911D94CA0>, 'Connection to geocode.arcgis.com timed out. (connect timeout=1)'))

In [ ]:
nashik3.to_csv('nashik3.csv')

In [ ]:
nashik4 = nashik.iloc[1500:2000]
nashik4['cordinates']= nashik4['address'].apply(nom.geocode)


nashik4['latitude'] = nashik4['cordinates'].apply(lambda x:x.latitude)
nashik4['longitude'] = nashik4['cordinates'].apply(lambda x:x.longitude)

In [ ]:
nashik4.to_csv('nashik4.csv')

In [ ]:
nashik5 = nashik.iloc[2000:2500]
nashik5['cordinates']= nashik5['address'].apply(nom.geocode)


nashik5['latitude'] = nashik5['cordinates'].apply(lambda x:x.latitude)
nashik5['longitude'] = nashik5['cordinates'].apply(lambda x:x.longitude)

In [ ]:
nashik5.to_csv('nashik5.csv')

In [ ]:
nashik6 = nashik.iloc[2500:3000]
nashik6['cordinates']= nashik6['address'].apply(nom.geocode)


nashik6['latitude'] = nashik6['cordinates'].apply(lambda x:x.latitude)
nashik6['longitude'] = nashik6['cordinates'].apply(lambda x:x.longitude)

In [ ]:
nashik6.to_csv('nashik6.csv')